# Universal References

Concerning the rvalue references (`&&`), there is one final specific behavior to know, and which occurs in the case of a function template having a `T` parameter, and an argument which is **very exactly** `T&&`. If the function is invoked with a temporary value, the inferred type for the argument is indeed `T&&`, but it is also legal to invoke the function with a stable value, and the inferred type for the argument is then `T&`.

So, within the context of a function template, an argument of type `T&&` can accept both a stable or a temporary value. Therefore `T&&` can be qualified as **universal reference**. The idea is to be able to write function patterns that know how to manipulate both stable and temporary values, and take advantage of the displacement possibilities when it is possible. Let's take an example:

In [1]:
#include<iostream>
template<typename T>
void foo(T&& t)
{ std::cout<<t<<std::endl ; }

If it is called with a temporary value, then `T` is deduced of the same type as the value :

In [2]:
{
    foo(42);            // Calls foo<int>(int&&)
    foo(3.14159);       // Calls foo<double>(double&&)
    foo(std::string("bonjour")); // Calls foo<std::string>(std::string&&)
}

42
3.14159
bonjour


On the other hand, if it is called with a non-temporary value, then `T` and the type of the argument are considered as classical references :

In [3]:
{
    int i=42;
    foo(i); // Calls foo<int&>(int&)
}

42


This makes it possible to have a single function template that accepts both temporary and non-temporary values, and able to perform a move rather than a copy when it is possible.

## std::forward

In the case of a function calling another function, the problem of preserving the temporary character of a value arises again. For ordinary functions, `std::move` did the trick. But within the framework of a function template and a universal reference, one would like to be able to preserve the temporary property of the temporary variables, but not to add this property to the stable variables. This is precisely the role of the `std::forward` function.

In [4]:
#include<iostream>
template<typename T>
void do_stuff(T&& t)
{
  std::cout<<"stuff for " ;
  foo(std::forward<T>(t));
}

In [5]:
{
    do_stuff(42);
    do_stuff(3.14159);
    do_stuff(std::string("hello"));
    
    int i=42;
    do_stuff(i);
}

stuff for 42
stuff for 3.14159
stuff for hello
stuff for 42


It is called a **perfect forwarding**, because the `foo` function receives an argument which is exactly the same type as the value initially passed to` do_stuff`.

© *CNRS 2021*  
*Assembled and written in french by David Chamont, translated by Karim Hasnaoui, this work is made available according to the terms of the*  
[*Creative Commons License - Attribution - NonCommercial - ShareAlike 4.0 International*](http://creativecommons.org/licenses/by-nc-sa/4.0/)